# Experiment 10: Selective Truncation — Condition Examples

This notebook shows the actual text for each experimental condition using real data from the dataset. No GPU needed.

In [ ]:
import os, sys, json, re
import numpy as np
from pathlib import Path
from collections import Counter

sys.path.insert(0, ".")
from lib.data import count_words

SEED = 42

# ---- Load MS MARCO (same reconstruction as Exp 01/02/etc.) ----
from datasets import load_dataset
ds = load_dataset("microsoft/ms_marco", "v1.1", split="validation")

samples = []
for item in ds:
    if len(samples) >= 1500:
        break
    passages = item.get('passages', {})
    ptexts = passages.get('passage_text', [])
    is_sel = passages.get('is_selected', [])
    query = item.get('query', '')
    answers = item.get('answers', [])
    well_formed = item.get('wellFormedAnswers', [])
    answer = None
    if well_formed and len(well_formed) > 0 and well_formed[0] not in ('[]', ''):
        answer = well_formed[0]
    elif answers and len(answers) > 0 and answers[0] != 'No Answer Present.':
        answer = answers[0]
    if not answer:
        continue
    for pt, sel in zip(ptexts, is_sel):
        wc = count_words(pt)
        if sel == 1 and 30 <= wc <= 300:
            samples.append({
                'passage': pt, 'query': query, 'answer': answer,
                'word_count': wc,
            })
            break

np.random.seed(SEED)
np.random.shuffle(samples)
samples = samples[:500]
del ds

STOP_WORDS = {
    'a', 'an', 'the', 'is', 'are', 'was', 'were', 'be', 'been', 'being',
    'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could',
    'should', 'may', 'might', 'can', 'shall', 'to', 'of', 'in', 'for',
    'on', 'with', 'at', 'by', 'from', 'as', 'into', 'through', 'during',
    'before', 'after', 'above', 'below', 'between', 'and', 'but', 'or',
    'not', 'no', 'if', 'then', 'than', 'so', 'up', 'out', 'about',
    'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those',
    'it', 'its', 'i', 'me', 'my', 'we', 'our', 'you', 'your', 'he',
    'him', 'his', 'she', 'her', 'they', 'them', 'their', 'how', 'when',
    'where', 'why', 'much', 'many', 'some', 'any', 'all', 'each',
    'does', 'also', 'just', 'more', 'most', 'very', 'too', 'only',
}

def extract_keywords(text):
    words = re.sub(r'[^\w\s]', '', text.lower()).split()
    return [w for w in words if w not in STOP_WORDS and len(w) > 2]

def make_surrogate_paraphrase(query):
    keywords = extract_keywords(query)
    return " ".join(keywords[::-1]) if keywords else query

def make_surrogate_from_doc(passage):
    content_words = extract_keywords(passage)
    if not content_words:
        return "information"
    counts = Counter(content_words)
    return " ".join(w for w, _ in counts.most_common(5))

def make_surrogate_template(passage):
    content_words = extract_keywords(passage)
    if not content_words:
        return "What is this about?"
    counts = Counter(content_words)
    top_word = counts.most_common(1)[0][0]
    return f"What is {top_word}?"

# Verify against checkpoint
def verify_checkpoint(exp_name):
    ckpt_path = Path(f"results/{exp_name}/checkpoint.json")
    if ckpt_path.exists():
        ckpt = json.loads(ckpt_path.read_text())
        results = ckpt.get('results', [])
        if results and results[0].get('query', '')[:50] == samples[0]['query'][:50]:
            print(f"  Checkpoint verification: MATCH ({exp_name})")
            return True
        elif results:
            print(f"  Checkpoint verification: MISMATCH ({exp_name})")
            print(f"    Checkpoint: {results[0].get('query', '')[:50]}")
            print(f"    Samples:    {samples[0]['query'][:50]}")
            return False
    else:
        print(f"  No checkpoint found for {exp_name}")
    return None

print(f"Loaded {len(samples)} MS MARCO samples (SEED={SEED})")
print(f"Sample 0 query: {samples[0]['query'][:70]}")


def show_sample(s, doc_key='passage', n=0):
    # Show sample info
    doc = s[doc_key]
    print(f"{'='*80}")
    print(f"SAMPLE {n}")
    print(f"{'='*80}")
    print(f"  Query:    {s['query']}")
    print(f"  Answer:   {s['answer']}")
    print(f"  Document: {doc[:100]}...")
    print(f"  Doc words: {len(doc.split())}")
    print()

def show_conditions(conditions, doc_text):
    # conditions: list of (name, description, encoder_prefix_text_or_None)
    # For bare conditions, encoder_prefix_text is None
    print(f"{'Condition':<30} {'Prefix':<14} {'Encoder input (first 70 chars)'}")
    print(f"{'-'*100}")
    for name, desc, prefix_text in conditions:
        if prefix_text is None:
            enc_preview = doc_text[:70]
            print(f"{name:<30} {'(none)':<14} {enc_preview}...")
        else:
            enc_text = prefix_text + "\n" + doc_text
            print(f"{name:<30} {str(len(prefix_text.split()))+'w':<14} {enc_text[:70]}...")
        if desc:
            print(f"  {'':>28} ^ {desc}")
    print()


show_sample(samples[0])

ex = samples[0]
query_kws = extract_keywords(ex['query'])
doc_kws = extract_keywords(ex['passage'])
query_kw = query_kws[0] if query_kws else "topic"
doc_kw = doc_kws[0] if doc_kws else "topic"

other_idx = (0 + 250) % len(samples)
rand_text = " ".join(samples[other_idx]['passage'].split()[:20])

print(f"  Query keyword: '{query_kw}'")
print(f"  Doc keyword:   '{doc_kw}'")
print()

print("SELECTIVE TRUNCATION: What if the decoder can see just ONE keyword from the prefix?")
print()

conditions = [
    ("bare", "Baseline", None),
    ("oracle_trunc", "Query, decoder sees doc only", ex['query']),
    ("oracle_full", "Query, decoder sees query+doc", ex['query']),
    ("oracle_kw_visible", f"Query, decoder sees doc + '{query_kw}' token", ex['query']),
    ("template_trunc", "'What is [kw]?', truncated", f"What is {doc_kw}?"),
    ("template_kw_visible", f"'What is [kw]?', decoder sees '{doc_kw}' token",
     f"What is {doc_kw}?"),
    ("pad_kw_trunc", f"'the the the {doc_kw}', truncated",
     f"the the the {doc_kw}"),
    ("pad_kw_kw_visible", f"'the the the {doc_kw}', decoder sees '{doc_kw}' token",
     f"the the the {doc_kw}"),
    ("random_trunc", "Random text, truncated", rand_text),
    ("keyword_only_visible", f"'{doc_kw}' as entire prefix, decoder sees it", doc_kw),
]
show_conditions(conditions, ex['passage'])

print("KEY QUESTION: Is there a sweet spot between full truncation and full visibility?")
print("  Full truncation (d=+0.408): decoder sees doc only")
print("  Full visibility (d=+0.345): decoder sees everything")
print("  Selective: decoder sees doc + just the keyword token")
